# Recomendador de artigos - Open Alex e GPT

In [ ]:
import requests
import pandas as pd
from datetime import datetime,timedelta,date
import os

pd.options.display.max_columns = 999

In [ ]:
def coletar_dados_e_salvar(data_inicial,data_fim):
    
    cursor = '*'
            
    contador = 1
    contador_erro = 0
    
    diretorio = f'datasets_{data_inicial}_to_{data_fim}'
    
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
    
    while cursor != None:
        url = f'https://api.openalex.org/works?filter=from_publication_date:{data_inicial},to_publication_date:{data_fim},type:Article&per-page=200&cursor={cursor}'

        
        try:
            requisicao = requests.get(url)
            pagina_com_resultados = requisicao.json()
            
        except (requests.exceptions.RequestException, ValueError) as e:
            contador_erro += 1 
            print(f"Erro na página {contador} (Erro {contador_erro}): {e}")
            continue 
        
        resultados = pagina_com_resultados.get('results', [])
        
        df = pd.DataFrame(resultados)
        parquet_arquivo = os.path.join(diretorio, f'registros_{data_inicial}_to_{data_fim}_{contador}.parquet')
        df.to_parquet(parquet_arquivo, index=False)
        
        cursor = pagina_com_resultados['meta'].get('next_cursor')
        
        contador += 1
    
    print(f"Total de erros: {contador_erro}")

In [ ]:
def concatenar_arquivos_parquet(folder_path):

    dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.parquet'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_parquet(file_path)
            dataframes.append(df)

    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    df_concatenado.to_parquet('df_concatenado.parquet',index=False)
    

In [ ]:
def extrair_concepts_scores(df):
    concept_data = []

    for concepts in df['concepts']:
        concept_scores = {}
        for concept in concepts:
            if concept['level'] < 2 and concept['score'] > 0.75:
                concept_scores[concept['display_name']] = concept['score']
        concept_data.append(concept_scores)

    df_concepts = pd.DataFrame(concept_data, index=df.index).fillna(0).round(4)

    df_final = pd.concat([df, df_concepts], axis=1)

    return df_final

In [ ]:
def get_source_name(primary_location):
    if primary_location is None:
        return None
    source_info = primary_location.get('source', None)
    
    if source_info is None:
        return None
    display_name = source_info.get('display_name', None)
    
    return display_name



def transformar_df(df):
    
    df = df.loc[:,['doi','title','publication_date','created_date','primary_location','open_access','concepts','cited_by_count']]
    
    df_filtrado = df.copy()
    
    df_filtrado.loc[:,'source_title'] = df_filtrado['primary_location'].apply(get_source_name)
    
    df_filtrado.loc[:,'open_access_status'] = df_filtrado['open_access'].apply(lambda x: x.get('is_oa',None))
    
    df_filtrado = extrair_concepts_scores(df_filtrado)
    
    df_filtrado.drop(columns=['primary_location','open_access','concepts'],inplace=True)
    
    return df_filtrado

In [ ]:
def ler_e_transformar_arquivos_diretorio(diretorio_input, diretorio_output):
    
    if not os.path.exists(diretorio_output):
        os.makedirs(diretorio_output)
    
    arquivos = [arquivo for arquivo in os.listdir(diretorio_input) if arquivo.endswith(".parquet")]
    
    try:
        for arquivo_parquet in arquivos:
            caminho_input = os.path.join(diretorio_input, arquivo_parquet)
            df_original = pd.read_parquet(caminho_input)  
            df_processado = transformar_df(df_original) 
            
            output_file = os.path.splitext(arquivo_parquet)[0] + ".parquet"
            caminho_output = os.path.join(diretorio_output, output_file)
            
            df_processado.to_parquet(caminho_output, index=False)

    except Exception as error:
        print(arquivo_parquet,error)

In [ ]:
def concatenar_arquivos_parquet(diretorio_input):

    dataframes = []

    for arquivo in os.listdir(diretorio_input):
        if arquivo.endswith('.parquet'):
            file_path = os.path.join(diretorio_input, arquivo)
            df = pd.read_parquet(file_path)
            dataframes.append(df)

    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    return df_concatenado

In [ ]:
data_inicial = '2023-09-22'
data_final = '2023-09-29'

coletar_dados_e_salvar(data_inicial,data_final)

Total de erros: 0


In [ ]:
diretorio_input = 'datasets_2023-09-22_to_2023-09-29'
diretorio_output = 'datasets_2023-09-22_to_2023-09-29_processados'

ler_e_transformar_arquivos_diretorio(diretorio_input,diretorio_output)

registros_2023-09-22_to_2023-09-29_307.parquet "None of [Index(['doi', 'title', 'publication_date', 'created_date', 'primary_location',\n       'open_access', 'concepts', 'cited_by_count'],\n      dtype='object')] are in the [columns]"


___

In [ ]:
concatenar_arquivos_parquet(diretorio_output).to_parquet('df_concatenado.parquet')

In [2]:
df_concatenado = pd.read_parquet('/home/franciscofoz/Documents/GitHub/recomendador-artigos-OpenAlex-GPT/df_concatenado.parquet')

In [13]:
concept = 'Psychology'

df_concatenado.query('`Psychology` > 0.75 ')[
    ['doi','title','publication_date','cited_by_count',concept]
    ].sort_values(by='Psychology',ascending=False)

,doi,title,publication_date,cited_by_count,Psychology
31924,https://doi.org/10.1037/mac0000135.supp,Supplemental Material for Positive and Negativ...,2023-09-21,0,0.9004
27850,https://doi.org/10.1037/mac0000135,Positive and negative vicarious memories in co...,2023-09-21,0,0.9000
3834,https://doi.org/10.1037/dev0001598,Attention biases for emotional facial expressi...,2023-09-21,0,0.8952
27866,https://doi.org/10.1037/fam0001156.supp,Supplemental Material for Parental Guilt and C...,2023-09-21,0,0.8907
47221,https://doi.org/10.1111/ajsp.12584,"Dispositional authenticity, facilitativeness, ...",2023-09-22,0,0.8875
...,...,...,...,...,...
3961,https://doi.org/10.1037/adb0000960,Network structures of internet gaming disorder...,2023-09-21,0,0.7514
25764,https://doi.org/10.15448/2179-8435.2023.1.43285,uso de jogos didáticos no ensino de Ciências d...,2023-09-26,0,0.7513
43949,https://doi.org/10.24854/jpu596,Frustrasi kebutuhan dasar psikologis memediasi...,2023-09-25,0,0.7510
44427,https://doi.org/10.2486/indhealth.2023-0028,Boredom and engagement at work: do they have d...,2023-09-26,0,0.7504


In [10]:
df_concatenado

doi  \
0              https://doi.org/10.3390/su151914071   
1      https://doi.org/10.24002/kinerja.v27i2.6574   
2              https://doi.org/10.3390/su151914114   
3            https://doi.org/10.3390/polym15193872   
4            https://doi.org/10.3390/polym15193873   
...                                            ...   
47948    https://doi.org/10.47458/slauga.2023.4.25   
47949    https://doi.org/10.47458/slauga.2023.4.26   
47950    https://doi.org/10.1177/00375497231198859   
47951         https://doi.org/10.12890/2023_004088   
47952     https://doi.org/10.3390/coatings13101677   

                                                   title publication_date  \
0      Reliability Analysis of Nuclear Power Plant El...       2023-09-22   
1      The Impact of Training and Transformational Le...       2023-09-24   
2      Spatio-Temporal Evolution and Action Path of E...       2023-09-23   
3      The Behavior of Polymeric Pipes in Drinking Wa...       2023-09-24   
4      Influence of Light Conditions on the Antibacte...       2023-09-24   
...                                                  ...              ...   
47948  Factors of burnout syndrome in the professiona...       2023-09-26   
47949  Akušerių profesinė savivoka: Lietuvos situacij...       2023-09-26   
47950  Freeway work zone effects on vehicular fuel co...       2023-09-26   
47951  “Can’t intubate can’t oxygenate” situation in ...       2023-09-26   
47952  Improving the Protection Performance of Waterb...       2023-09-25   

      created_date  cited_by_count  \
0       2023-09-25               0   
1       2023-09-25               0   
2       2023-09-25               0   
3       2023-09-25               0   
4       2023-09-25               0   
...            ...             ...   
47948   2023-09-27               0   
47949   2023-09-27               0   
47950   2023-09-27               0   
47951   2023-09-27               0   
47952   2023-09-27               0   

                                            source_title  open_access_status  \
0                                         Sustainability                True   
1             Kinerja: Journal of Business and Economics                True   
2                                         Sustainability                True   
3                                               Polymers                True   
4                                               Polymers                True   
...                                                  ...                 ...   
47948                        Slauga. Mokslas ir praktika                True   
47949                        Slauga. Mokslas ir praktika                True   
47950                                         Simulation               False   
47951  European Journal of Case Reports in Internal M...                True   
47952                                           Coatings                True   

       Reliability engineering  Nuclear power plant  Fault (geology)  \
0                       0.7687               0.6362           0.5747   
1                       0.0000               0.0000           0.0000   
2                       0.0000               0.0000           0.0000   
3                       0.0000               0.0000           0.0000   
4                       0.0000               0.0000           0.0000   
...                        ...                  ...              ...   
47948                      NaN                  NaN              NaN   
47949                      NaN                  NaN              NaN   
47950                      NaN                  NaN              NaN   
47951                      NaN                  NaN              NaN   
47952                      NaN                  NaN              NaN   

       Failure mode and effects analysis  Transformational leadership  \
0                                 0.5233                       0.0000   
1                                 0.0000    

In [13]:
df_concatenado.sort_values(by='Library science',ascending=False)[['doi','title','publication_date',
                                                                  'cited_by_count','Library science']].head(10)

,doi,title,publication_date,cited_by_count,Library science
8804,https://doi.org/10.33463/2687-122x.2023.18(1-4...,Review of the manuscript of the bibliographic ...,2023-09-22,0,0.6488
14349,https://doi.org/10.11114/ijce.v6i2.6413,Reviewer Acknowledgements for International Jo...,2023-09-24,0,0.6480
14238,https://doi.org/10.4103/1673-5374.385863,Neurovascular unit on a chip: the relevance an...,2023-09-22,0,0.6249
26319,https://doi.org/10.1080/15420353.2023.2219072,"Here Today, Here Tomorrow: A Case Study of Rev...",2023-09-25,0,0.6063
11071,https://doi.org/10.1021/acs.organomet.3c00352,Irina Beletskaya: Chemistry Excellence in Scie...,2023-09-25,0,0.6031
3964,https://doi.org/10.1021/acs.jpcb.3c05596,Tribute to Xiaoliang Sunney Xie,2023-09-21,0,0.5905
21513,https://doi.org/10.1002/fgc.33264,Doctoral Dissertation Research Improvement Gra...,2023-09-21,0,0.5902
2896,https://doi.org/10.1002/jmri.29018,Editorial for “Longitudinal Metabolite Changes...,2023-09-22,0,0.5893
21466,https://doi.org/10.1002/fgc.33243,Digital Humanities Advancement Grants (NEH),2023-09-21,0,0.5853
42284,https://doi.org/10.1002/fgc.33267,Postdoctoral Research Fellowships in Biology (...,2023-09-21,0,0.5828
